In [24]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install fsspec
!pip install gcsfs

In [25]:
#BIBLIOTECAS
!pip install pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, LongType
from pyspark.sql.window import Window
import pandas as pd
from pyspark.sql.functions import monotonically_increasing_id 

In [26]:
import os
from google.cloud import storage

In [27]:
#CONFIGURAR A SPARKSESSION

spark = (SparkSession.builder\
        .master("local")\
        .appName("ProjetoFinal_Pyspark")\
        .config("spark.ui.port", "4050")\
        .getOrCreate())

spark

In [28]:
# Abrindo o arquivo
serviceaccount = '/content/Chave_Ingestao_Apache.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = serviceaccount

df1 = spark\
    .read\
    .format('csv')\
    .option('header', 'True')\
    .option('inferschema','False')\
    .option('delimiter', ',')\
    .load('/content/saida_dados_Projeto_Final_ZAP_Pandas.csv')


df1.show(5)
df1.printSchema() 

+--------------------+-----------+--------------+-------------+--------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+-------------+-----------+----------+-----------+------------+-------------+-----+---------------------+----------+
|        nome_empresa|tipo_imovel| cidade_imovel|site_pesquisa| estado_imovel|qtde_banheiros|qtde_quartos|tipo_contrato|mobilhado|academia|desocupado|garagem|area_lazer|piscina|plataforma_digital|preco_condominio|preco_periodo|valor_aluguel|valor_venda|iptu_anual|qtde_suites|quadra_tenis|area_total_m2|andar|residencial_comercial|      data|
+--------------------+-----------+--------------+-------------+--------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+-------------+-----------+----------+-----------+------------+-------------+-----+-----

In [29]:
df1 = df1.select("*").withColumn("id", monotonically_increasing_id())
df1.show()

+--------------------+-----------+--------------------+-------------+-----------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+------------------+-----------+----------+-----------+------------+-------------+-----+---------------------+----------+---+
|        nome_empresa|tipo_imovel|       cidade_imovel|site_pesquisa|    estado_imovel|qtde_banheiros|qtde_quartos|tipo_contrato|mobilhado|academia|desocupado|garagem|area_lazer|piscina|plataforma_digital|preco_condominio|preco_periodo|     valor_aluguel|valor_venda|iptu_anual|qtde_suites|quadra_tenis|area_total_m2|andar|residencial_comercial|      data| id|
+--------------------+-----------+--------------------+-------------+-----------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+------------------+-----------+----------+

In [30]:
# Alterando nomes das colunas "valoraluguel", "valor_venda" e "qdte_quartos"
df1 = df1.withColumnRenamed("valoraluguel", "valor_aluguel")
df1.show(1)

+--------------------+-----------+-------------+-------------+-------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+-------------+-----------+----------+-----------+------------+-------------+-----+---------------------+----------+---+
|        nome_empresa|tipo_imovel|cidade_imovel|site_pesquisa|estado_imovel|qtde_banheiros|qtde_quartos|tipo_contrato|mobilhado|academia|desocupado|garagem|area_lazer|piscina|plataforma_digital|preco_condominio|preco_periodo|valor_aluguel|valor_venda|iptu_anual|qtde_suites|quadra_tenis|area_total_m2|andar|residencial_comercial|      data| id|
+--------------------+-----------+-------------+-------------+-------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+-------------+-----------+----------+-----------+------------+-------------+-----+---

In [31]:
#Drop coluna "descocupado"
df1.drop('desocupado')
df1.show(5)

+--------------------+-----------+--------------+-------------+--------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+-------------+-----------+----------+-----------+------------+-------------+-----+---------------------+----------+---+
|        nome_empresa|tipo_imovel| cidade_imovel|site_pesquisa| estado_imovel|qtde_banheiros|qtde_quartos|tipo_contrato|mobilhado|academia|desocupado|garagem|area_lazer|piscina|plataforma_digital|preco_condominio|preco_periodo|valor_aluguel|valor_venda|iptu_anual|qtde_suites|quadra_tenis|area_total_m2|andar|residencial_comercial|      data| id|
+--------------------+-----------+--------------+-------------+--------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+-------------+-----------+----------+-----------+------------+-------------+---

In [32]:
#Quantidade de cada tipo de imóvel presente no dataframe em ordem decrescente
df1.groupBy(F.col("tipo_imovel")).count().sort("count", ascending=False).show()

+------------------+-----+
|       tipo_imovel|count|
+------------------+-----+
|       apartamento| 7440|
|              casa| 6768|
|casa-de-condominio| 6720|
|              flat| 5208|
|         cobertura| 4763|
|         quitinete| 4392|
|      casa-de-vila|  354|
|            studio|  127|
+------------------+-----+



In [33]:
# Descrição do imovel por tipo e cômodos
df1.select(F.col("tipo_imovel"),
           F.col("qtde_quartos"), 
           F.col("qtde_suites"),
           F.col("qtde_banheiros"),  
           F.col("quadra_tenis"), 
           F.col("piscina"), 
           F.col("area_lazer"), 
           F.col("garagem")).show(10)

+-----------+------------+-----------+--------------+------------+-------+----------+-------+
|tipo_imovel|qtde_quartos|qtde_suites|qtde_banheiros|quadra_tenis|piscina|area_lazer|garagem|
+-----------+------------+-----------+--------------+------------+-------+----------+-------+
|apartamento|           2|          0|             1|         Não|    Não|       Não|      0|
|apartamento|           3|          1|             3|         Não|    Não|       Não|      1|
|apartamento|           3|          1|             3|         Não|    Não|       Não|      2|
|apartamento|           1|          0|             2|         Não|    Sim|       Não|      1|
|apartamento|           2|          2|             3|         Não|    Sim|       Sim|      2|
|apartamento|           2|          1|             1|         Não|    Sim|       Não|      1|
|apartamento|           1|          1|             2|         Não|    Sim|       Não|      1|
|apartamento|           2|          1|             2|       

In [34]:
#Quantidade de imóveis anunciados, em ordem decrescente, em cidades do estado de São Paulo
df1.groupBy(F.col("cidade_imovel")).count().sort("count", ascending=False).show()

+--------------------+-----+
|       cidade_imovel|count|
+--------------------+-----+
|           São Paulo|12533|
|      Rio de Janeiro| 3639|
|            Campinas| 2327|
|        Porto Alegre| 1473|
|      Belo Horizonte|  873|
|            Curitiba|  798|
|      Ribeirão Preto|  716|
|             Barueri|  634|
| Santana de Parnaíba|  546|
|       Florianópolis|  399|
|             Niterói|  381|
|         Santo André|  375|
| São José dos Campos|  345|
|São Bernardo do C...|  316|
|            Valinhos|  302|
|            Sorocaba|  300|
|             Jundiaí|  296|
|           Fortaleza|  296|
|            Londrina|  280|
|             Vinhedo|  260|
+--------------------+-----+
only showing top 20 rows



In [35]:
#Exibindo "studios" no estado de São Paulo
df1.where((F.col("tipo_imovel") == "studio") & (F.col("estado_imovel") == "São Paulo")).show()

+--------------------+-----------+--------------+-------------+-------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+-------------+-----------+----------+-----------+------------+-------------+-----+---------------------+----------+-----+
|        nome_empresa|tipo_imovel| cidade_imovel|site_pesquisa|estado_imovel|qtde_banheiros|qtde_quartos|tipo_contrato|mobilhado|academia|desocupado|garagem|area_lazer|piscina|plataforma_digital|preco_condominio|preco_periodo|valor_aluguel|valor_venda|iptu_anual|qtde_suites|quadra_tenis|area_total_m2|andar|residencial_comercial|      data|   id|
+--------------------+-----------+--------------+-------------+-------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+-------------+-----------+----------+-----------+------------+-------------+--

In [36]:
#Imóveis mobilhados no estado de Minas Gerais com 2 ou mais quartos

filtroMG = F.col("estado_imovel") == "Minas Gerais"
filtroMobS = F.col("mobilhado") == "Sim"
filtro2q = F.col("qtde_quartos") >= 2

df1.select(F.col("tipo_imovel"),
          F.col("qtde_quartos"),
          F.col("cidade_imovel"),
          F.col("mobilhado")).filter(filtroMG & filtroMobS & filtro2q).show()

+------------------+------------+--------------+---------+
|       tipo_imovel|qtde_quartos| cidade_imovel|mobilhado|
+------------------+------------+--------------+---------+
|       apartamento|           2|     Nova Lima|      Sim|
|       apartamento|           3|Belo Horizonte|      Sim|
|       apartamento|           2|Belo Horizonte|      Sim|
|       apartamento|           5|        Viçosa|      Sim|
|       apartamento|           2|Belo Horizonte|      Sim|
|       apartamento|           3|Belo Horizonte|      Sim|
|       apartamento|           3|Belo Horizonte|      Sim|
|       apartamento|           2|Belo Horizonte|      Sim|
|       apartamento|           3|     Nova Lima|      Sim|
|       apartamento|           3|Belo Horizonte|      Sim|
|       apartamento|           3|     Nova Lima|      Sim|
|       apartamento|           3|Belo Horizonte|      Sim|
|       apartamento|           3|       Uberaba|      Sim|
|       apartamento|           3|Belo Horizonte|      Si

In [37]:
#Quantidade de imóveis "encontrados" por cada site de pesquisa
df1.groupBy(F.col("site_pesquisa")).count().sort("count", ascending=False).show()

+-------------------+-----+
|      site_pesquisa|count|
+-------------------+-----+
|             GOOGLE|35252|
|           GRUPOZAP|  333|
|      Não Informado|  138|
|          PUBLISHER|   32|
|GeoPointSource_NONE|   17|
+-------------------+-----+



In [38]:
#Valor máximo de aluguel em cada tipo de imóvel
(df1.groupby(F.col("tipo_imovel")).agg(
    F.max("valor_aluguel").alias("valor_max_aluguel"),
    F.max("valor_venda").alias("Valor_max_venda")).show())

+------------------+-----------------+---------------+
|       tipo_imovel|valor_max_aluguel|Valor_max_venda|
+------------------+-----------------+---------------+
|       apartamento|            999.0|        990.000|
|              casa|            995.0|        995.000|
|casa-de-condominio|            990.0|        999.000|
|      casa-de-vila|            980.0|      9.800.000|
|         cobertura|            999.0|        999.900|
|              flat|            995.0|        996.400|
|         quitinete|            999.0|         98.000|
|            studio|            950.0|        640.000|
+------------------+-----------------+---------------+



In [39]:
#Quantidade media de quartos por tipo de imovel
(df1.groupBy(F.col("tipo_imovel")).agg(F.round(F.mean("qtde_quartos"), 2).alias("media_qtde_quartos"))).show()

+------------------+------------------+
|       tipo_imovel|media_qtde_quartos|
+------------------+------------------+
|      casa-de-vila|              2.24|
|            studio|              0.92|
|       apartamento|              2.11|
|              flat|              1.12|
|casa-de-condominio|              3.45|
|         quitinete|               0.9|
|              casa|              2.75|
|         cobertura|              3.21|
+------------------+------------------+



In [40]:
#Filtro por cidades iniciadaas com J
df1.filter("cidade_imovel like 'J%' ").show()

+--------------------+-----------+-------------+-------------+--------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+-------------+-----------+----------+-----------+------------+-------------+-----+---------------------+----------+---+
|        nome_empresa|tipo_imovel|cidade_imovel|site_pesquisa| estado_imovel|qtde_banheiros|qtde_quartos|tipo_contrato|mobilhado|academia|desocupado|garagem|area_lazer|piscina|plataforma_digital|preco_condominio|preco_periodo|valor_aluguel|valor_venda|iptu_anual|qtde_suites|quadra_tenis|area_total_m2|andar|residencial_comercial|      data| id|
+--------------------+-----------+-------------+-------------+--------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+-------------+-----------+----------+-----------+------------+-------------+-----+

In [41]:
w0 = Window.partitionBy(F.col("nome_empresa")).orderBy("preco_condominio")
(df1.withColumn("row_number", F.row_number().over(w0)).select(F.col("nome_empresa"),
                                                              F.col("tipo_imovel"),
                                                              F.col("cidade_imovel"), 
                                                              F.col("residencial_comercial"),
                                                              F.col("preco_condominio"))).show()

+--------------------+-----------+--------------------+---------------------+----------------+
|        nome_empresa|tipo_imovel|       cidade_imovel|residencial_comercial|preco_condominio|
+--------------------+-----------+--------------------+---------------------+----------------+
|ADI Assessoria e ...|apartamento|           São Paulo|          Residencial|              50|
|   San Remos Imóveis|apartamento|       Florianópolis|          Residencial|           1.961|
|Família Bacellar ...|apartamento|      Rio de Janeiro|          Residencial|             835|
|Next Soluções Imo...|apartamento|            Campinas|          Residencial|         134.437|
|QUINTO ANDAR SERV...|apartamento|           São Paulo|          Residencial|             536|
|FLAVIA DANIELLE P...|apartamento|      Rio de Janeiro|          Residencial|             809|
|QUINTO ANDAR SERV...|apartamento|           São Paulo|          Residencial|             880|
|LELLO IMÓVEIS TAT...|apartamento|           São P

In [42]:
client = storage.Client()
bucket = client.get_bucket('projeto-final-grupo09')
     
bucket.blob('saida_dados/Projeto_Final_ZAP_Pyspark.csv').upload_from_string(df1.toPandas().to_csv(index=False), 'text/csv')